In [1]:
import pandas as pd
from sklearn import tree, preprocessing, model_selection, ensemble, feature_selection
import os
from mlxtend import classifier as mlxClassifier
titanic_train = pd.read_csv('/Users/bharath/Downloads/titanic/train.csv')
titanic_train.info()
titanic_test = pd.read_csv('/Users/bharath/Downloads/titanic/test.csv')
titanic_test.info()

mean_imputer = preprocessing.Imputer()
mean_imputer.fit(titanic_train[['Age','Fare']])
titanic_test['Survived'] = None
titanic = pd.concat([titanic_train, titanic_test])

titanic[['Age','Fare']] = mean_imputer.transform(titanic[['Age','Fare']])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float

/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


In [2]:
def split_title(name):
    return name.split(',')[1].split('.')[0].strip()

titanic['Title'] = titanic['Name'].map(split_title)


In [3]:
    def Age_group(age):
        if (age >=0 and age <=10):
            return 'Child'
        elif (age >10 and age <= 25):
              return 'Young'
        elif (age >25 and age <= 50):
              return 'Middle'
        else:
            return 'Old'    

In [4]:
titanic['Age1'] = titanic['Age'].map(Age_group)

In [5]:
titanic['FamilySize'] = titanic['Parch'] + titanic['SibSp'] + 1

In [6]:
def family_size(size):
    if (size ==1):
        return 'Single'
    elif (size >1 and size <= 3):
          return 'Small'
    elif (size >3 and size <=6):
          return 'Medium'
    else:
        return 'large'

In [7]:
    titanic['Family_type'] = titanic['FamilySize'].map(family_size)

In [8]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 16 columns):
Age            1309 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1309 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null object
Ticket         1309 non-null object
Title          1309 non-null object
Age1           1309 non-null object
FamilySize     1309 non-null int64
Family_type    1309 non-null object
dtypes: float64(2), int64(5), object(9)
memory usage: 173.9+ KB


In [9]:
titanic1 = pd.get_dummies(titanic, columns=['Pclass','Sex','Embarked','Age1','Family_type','Title'])

In [10]:
titanic1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 44 columns):
Age                   1309 non-null float64
Cabin                 295 non-null object
Fare                  1309 non-null float64
Name                  1309 non-null object
Parch                 1309 non-null int64
PassengerId           1309 non-null int64
SibSp                 1309 non-null int64
Survived              891 non-null object
Ticket                1309 non-null object
FamilySize            1309 non-null int64
Pclass_1              1309 non-null uint8
Pclass_2              1309 non-null uint8
Pclass_3              1309 non-null uint8
Sex_female            1309 non-null uint8
Sex_male              1309 non-null uint8
Embarked_C            1309 non-null uint8
Embarked_Q            1309 non-null uint8
Embarked_S            1309 non-null uint8
Age1_Child            1309 non-null uint8
Age1_Middle           1309 non-null uint8
Age1_Old              1309 non-null uint8
Age1_Y

In [11]:
titanic2 = titanic1.drop(['Age','Cabin','Name','PassengerId','Ticket','Survived'],1) 
# why 'Parch','SibSp' are not dropped ?
x_train = titanic2[0:titanic_train.shape[0]]
y_train = titanic_train['Survived']

dt_rand = ensemble.RandomForestClassifier(n_estimators=5)
dt_rand.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [12]:
features = pd.DataFrame({'feature':x_train.columns, 'importance': dt_rand.feature_importances_ })
features.sort_values(by=['importance'],ascending=True, inplace=True)
#features.set_index('feature', inplace=True)
features.plot(kind='barh',figsize=(30,30))
x_train.shape
fs = feature_selection.SelectFromModel(dt_rand, threshold='median', prefit=True)
x_train1 = fs.transform(x_train)
x_train1.shape

x_test = titanic2[titanic_train.shape[0]:]
x_test1 = fs.transform(x_test)
x_test1.shape

(418, 19)

In [14]:
#rf1 = ensemble.RandomForestClassifier(random_state=100)
#ada2 = ensemble.AdaBoostClassifier(random_state=100)

#super_dt = tree.DecisionTreeClassifier(random_state=100)

#stack_estimator = mlxClassifier.StackingClassifier(classifiers=[rf1,ada2], meta_classifier=super_dt)

#stack_grid = {'randomforestclassifier__n_estimator': [5,10],
#              'adaboostclassifier__n_estimator': [10,50],
#              'meta-decisiontreeclassifier__min_samples_split': [2,3]}

#grid_stack_estimator = model_selection.GridSearchCV(stack_estimator,stack_grid,cv=10 )
rf1 = ensemble.RandomForestClassifier(random_state=100)
ada2 = ensemble.AdaBoostClassifier(random_state=100)

dtSuper = tree.DecisionTreeClassifier(random_state=100)

stack_estimator = mlxClassifier.StackingClassifier(classifiers=[rf1, ada2], meta_classifier=dtSuper) #, store_train_meta_features=True)
stack_grid = {'randomforestclassifier__n_estimators': [5, 10],
            'adaboostclassifier__n_estimators': [10, 50],
            'meta_classifier__min_samples_split': [2, 3]}

grid_stack_estimator = model_selection.GridSearchCV(stack_estimator, stack_grid, cv=10)

#Random_estimator = ensemble.RandomForestClassifier(random_state=100, oob_score=True)
#random_grid = {'n_estimators': list(range(10,21,10))}

In [15]:
grid_stack_estimator.fit(x_train1, y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=StackingClassifier(average_probas=False,
          classifiers=[RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_...ures=False, use_clones=True,
          use_features_in_secondary=False, use_probas=False, verbose=0),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'randomforestclassifier__n_estimators': [5, 10], 'adaboostclassifier__n_estimators': [10, 50], 'meta_classifier__min_samples_split': [2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [18]:
grid_stack_estimator.best_estimator_

StackingClassifier(average_probas=False,
          classifiers=[RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_f...thm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=10, random_state=100)],
          drop_last_proba=False,
          meta_classifier=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=100,
            splitter='best'),
          store_train_meta_features=False, use_clones=True,
          use_features_in_secondary=False, use_probas=False, verbose=0)

In [19]:
grid_stack_estimator.best_score_

0.8024691358024691

In [20]:
grid_stack_estimator.score(x_train1, y_train)

0.9292929292929293

In [21]:
titanic_test['Survived'] = grid_stack_estimator.predict(x_test1)

In [22]:
os.chdir('/Users/bharath/Downloads/titanic')

In [23]:
titanic_test.to_csv('M12 with RF+FE+EDA+Stacking.csv', columns= ['PassengerId', 'Survived'], index=False)